# Guardian - Iteration 8

## Import Packages

In [6]:
# load .env File in Environment
from dotenv import load_dotenv
# operating system interfaces - get Environment-variables
import os
# Connect to API
import requests
# create Pandas Data Frame
import pandas as pd
import datetime as dt
from functools import wraps
load_dotenv()

True

### Set Params

In [7]:
iteration = 8

content_url = "https://content.guardianapis.com/search"
all_content_params = {
    "page-size": "50",
    "show-fields": "all",
    "show-tags": "all",
    "from-date": "2019-02-01",
    "to-date": "2019-07-01",
    "order-by": "newest",
    "api-key": os.getenv("GUARDIAN_API_KEY1"),
}


In [8]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

### Create DataFrame function


In [9]:
def get_df(url: str, params: dict):
        all_results = []
        current_page = 1
        total_pages = 1
        while current_page <= total_pages:
                tic = dt.datetime.now()
                params["page"] = current_page
                try:
                        r = requests.get(url, params)
                        all_results = all_results + r.json()["response"]["results"]
                        r.raise_for_status()
                except Exception as err:
                        SystemExit(err)
                if current_page == 1:
                        print("---- API STATUS ---- ")
                        print("status",  r.json()["response"]["status"])
                        total_pages = r.json()['response']['pages']
                        print("URL: ", r.url)
                        print("status",  r.json()["response"]["status"])
                        print("total",  r.json()["response"]["total"])
                        print("startIndex",  r.json()["response"]["startIndex"])
                        print("pageSize",  r.json()["response"]["pageSize"])
                        print("pages",  r.json()["response"]["pages"])
                        print("orderBy",  r.json()["response"]["orderBy"])
                        print("---- RUNTIME STATUS ---- ")

                time_taken = str(dt.datetime.now() - tic)
                print(f"({current_page}/{total_pages}) in {time_taken}s")
                        
                current_page += 1

        return pd.DataFrame(all_results)

### API Request

In [10]:
init_df = get_df(content_url, all_content_params)

---- API STATUS ---- 
status ok
URL:  https://content.guardianapis.com/search?page-size=50&show-fields=all&show-tags=all&from-date=2019-02-01&to-date=2019-07-01&order-by=newest&api-key=7e9dff14-114c-40a2-bb71-a816ac25ee64&page=1
status ok
total 33006
startIndex 1
pageSize 50
pages 661
orderBy newest
---- RUNTIME STATUS ---- 
(1/661) in 0:00:10.325418s
(2/661) in 0:00:03.443841s
(3/661) in 0:00:06.653419s
(4/661) in 0:00:07.781467s
(5/661) in 0:00:08.488456s
(6/661) in 0:00:05.177144s
(7/661) in 0:00:04.142663s
(8/661) in 0:00:09.840609s
(9/661) in 0:00:07.227185s
(10/661) in 0:00:10.308671s
(11/661) in 0:00:07.286105s
(12/661) in 0:00:13.744133s
(13/661) in 0:00:12.449213s
(14/661) in 0:00:06.045153s
(15/661) in 0:00:21.289129s
(16/661) in 0:00:09.539864s
(17/661) in 0:00:07.551807s
(18/661) in 0:00:10.763538s
(19/661) in 0:00:07.834988s
(20/661) in 0:00:04.927457s
(21/661) in 0:00:04.575693s
(22/661) in 0:00:06.751033s
(23/661) in 0:00:09.455611s
(24/661) in 0:00:03.645534s
(25/661) i

In [11]:
@log_step
def init_pipeline(df):
    return df.copy()

@log_step
def unfold_columns(df):
    dict_cols = ["fields"] 
    for col in dict_cols:
        new_df = pd.DataFrame()
        new_df[col] = df[col]#.apply(lambda x: eval(x))
        add_cols_df = pd.json_normalize(new_df[col])
        df = pd.concat([df, add_cols_df], axis=1)
    
    # Tags extraction
    
    #df["tags"] = df["tags"].apply(lambda x: eval(x))
    df['tagWebTitle'] = df['tags'].map(lambda x:[i['webTitle'] for i in x])
    df['tagId'] = df['tags'].map(lambda x:[i['id'] for i in x])
    df = df.drop(columns="tags")
    return df.drop(columns=dict_cols)
@log_step
def drop_columns(df):
    df = df.drop(columns=[
        "body",
        "webUrl",
        "apiUrl",
        "sectionId",
        "pillarId",
        "showAffiliateLinks",
        "bylineHtml",
        "standfirst",
        "shouldHideReaderRevenue",
        "isInappropriateForSponsorship",
        "shortUrl",
        "isPremoderated",
        "charCount",
        "shouldHideAdverts",
        "showInRelatedContent",
        "lang",
        "main",
        "thumbnail"
        
    ])
    return df

@log_step
def create_csv(df):
    df.to_csv(f"../../data/partial/articles_partial_{iteration}.csv.zip", compression="zip")
    return df

In [12]:
articles = (
    init_df
        .pipe(init_pipeline)
        .pipe(unfold_columns)
        .pipe(drop_columns)
        .pipe(create_csv)
)

init_pipeline:
 shape=(33006, 13) took 0:00:00.009709s

unfold_columns:
 shape=(33006, 49) took 0:00:08.273957s

drop_columns:
 shape=(33006, 31) took 0:00:00.018976s

create_csv:
 shape=(33006, 31) took 0:00:25.685571s

